In [1]:
from numpy import pi, cos, sin, sqrt
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon, Point
import pandas as pd
from math import modf
import json
from random import uniform
import requests
from numpy import mean

hex_width = 50
city_radius = 0.20 # in lat-long degrees

hex_size = city_radius/(2*hex_width)

washington_y = 38.904978
washington_x = -77.039658


atlanta_y = 33.7490
atlanta_x = -84.3880


In [2]:
# df = gpd.GeoDataFrame(pd.read_pickle('../transit/points_pickle'),crs={'init' :'epsg:3857'})

In [3]:
def axial_to_cube(x,y):
    z = -x -y
    return (x,y,z)

In [4]:
key_path = "C:\\Users\\John\\Documents\\notGitHub\\API_key.txt"
key_file = open(key_path, 'r')
api_key = key_file.readline()

In [5]:
string1 = "https://mobilityscore.transitscreen.io/api"
string2 = "/v1/locations.json?coordinates="
string3 = "/data/mobility&key="
string4 = "&geojson=true?fields=mobilityScore"

def api_string(coords):
    new_string = string1+string2+coords+string3+api_key+string4
    return new_string

In [6]:
class hexagon:
    def __init__(self, coords, city_center=(atlanta_y, atlanta_x), size=hex_size):
        # there will be no coordinate reference system at the 
        # hex level, that needs to be handled by the grid
        self.x_loc = float(coords[0])
        self.y_loc = float(coords[1])
        self.z_loc = float(coords[2])
        self.city_center = city_center
        self.size = size
        self.self_center()
        self.create_vertices()
        self.points=[]
        self.scores=[]
        self.bikeshares=[]
        self.carshares=[]
        self.masstransits=[]
        self.ridehailings=[]
    def self_center(self):
        self.x_center = self.size * 1.5 * self.x_loc
        self.y_center = self.size * sqrt(3) * (self.y_loc + self.x_loc/2)        
    def create_vertices(self):
        # I am only interested in making "flat top" layouts
        self.vertices = tuple(self.vertex(n) for n in range(6))
    def vertex(self, n):
        angle = n*pi/3
        i = self.size*cos(angle) + self.x_center + self.city_center[1]
        j = self.size*sin(angle) + self.y_center + self.city_center[0]
        return(i,j)
    
    def random_points(self, n):
        for i in range(n):
            angle = uniform(0,2*pi)
            distance = uniform(0,self.size*0.9)
            self.points.append(
                str(distance*cos(angle)+self.city_center[0])+
                ","+
                str(distance*sin(angle)+self.city_center[1]))
            
    def api_scrape(self):
        for coord in self.points:
            request_string = api_string(coord)
            result = requests.get(request_string)
            
            if result.status_code == 200:
                self.scores.append(result.json()['data']['mobilityScore']['score'])
                breakdown = result.json()['data']['mobilityScore']['scoreBreakdown']
                self.bikeshares.append(float(breakdown['bikeshare'].strip('%'))/100.0)
                self.carshares.append(float(breakdown['carshare'].strip('%'))/100.0)
                self.masstransits.append(float(breakdown['masstransit'].strip('%'))/100.0)
                self.ridehailings.append(float(breakdown['ridehailing'].strip('%'))/100.0)
            else:
                print('foobar')
            
        self.score = mean(self.scores)
        self.bikeshare = mean(self.bikeshares)
        self.carshare = mean(self.carshares)
        self.masstransit = mean(self.masstransits)
        self.ridehailing = mean(self.ridehailings)

In [7]:
## for testing purposes

# hex = hexagon((0,0,0))
# hex.random_points(5)
# hex.points
# hex.api_scrape()
# hex.score, hex.bikeshares, hex.carshare, hex.masstransit, hex.ridehailing

In [8]:


def xy_generator(center,grid,size):
    cx = grid[0] * 1.5 * size + center[1] # using lat-long convention for the city centers so x is the second value in center but not grid
    cy = size * sqrt(3) * ( grid[1] + grid[0]/2 ) + center[0]
    x=[]
    y=[]
    
    for n in range(6):
        angle=n*pi/3
        x.append(size*cos(angle)+cx)
        y.append(size*sin(angle)+cy)
    return (x,y)

xy_generator((39,-77),(0,0),0.025)



([-76.975, -76.9875, -77.0125, -77.025, -77.0125, -76.9875],
 [39.0,
  39.02165063509461,
  39.02165063509461,
  39.0,
  38.97834936490539,
  38.97834936490539])

In [55]:
from tqdm import tqdm

class grid_axial:
    def __init__(self, width, size=0.0025, coordinates = (38.904978, -77.039658)):
        self.width=width
        self.size=size
        self.coordinates = coordinates
        self.hex_dataframe_creator()
        self.polygons =[]
    def set_crs(self, crs):
        pass
    def make_ring(self,n):
        ring = []
        for m in range(1,n+1):
            ring.append(((n,-m,-n+m),m))
            ring.append(((-m,-n+m,n),m))
            ring.append(((-n+m,n,-m),m))
            ring.append(((-n,m, n-m),m))
            ring.append(((m,n-m,-n),m))
            ring.append(((n-m,-n,m),m))
        return ring
    
    def fill_grid(self):
        hex_list = [((0,0,0),0)]
        for n in range(1,self.width+1):
            hex_list+=self.make_ring(n)
        return hex_list
        
    def y_generator(self, grid):
        center = self.coordinates
        size = self.size
        cy = size * sqrt(3) * ( grid[1] + grid[0]/2 ) + center[0]
        # using lat-long convention for the city centers so x is the second value in center but not grid
        y=[]
        for n in range(6):
            angle=n*pi/3
            y.append(size*sin(angle)+cy)
        return y
    
    def x_generator(self,grid):
        center = self.coordinates
        size = self.size
        cx = 1.5 * float(grid[0]) * size + center[1] 
        # using lat-long convention for the city centers so x is the second value in center but not grid
        x=[]

        for n in range(6):
            angle=n*pi/3
            x.append(size*cos(angle)+cx)
        return x
    
    def hex_dataframe_creator(self):
        self.hexes = gpd.GeoDataFrame(self.fill_grid(), columns = ['hexagon','radius'])
        self.hexes['y'] = self.hexes.hexagon.apply(self.y_generator)
        self.hexes['x'] = self.hexes.hexagon.apply(self.x_generator)
        
    def check_randoms(self,n):
        hex_list = self.hexes['hexagon'].values[:1]
        for next in tqdm(hex_list):
            hex = hexagon(coords=next, city_center=self.coordinates, size=self.size)
            hex.random_points(n)
            hex.api_scrape()
            self.polygons.append(hex)
                          
    def poly_wag(self):
        df = self.hexes
        
        df['poly'] = self.polygons
        
        df['transit_score'] = df['poly'].apply(lambda x: x.score)
        df['bike_score'] = df['poly'].apply(lambda x: x.bikeshare)
        df['carshare_score'] = df['poly'].apply(lambda x: x.carshare)
        df['mass_transit_score'] = df['poly'].apply(lambda x: x.masstransit)
        df['ride_hail_score']= df['poly'].apply(lambda x: x.ridehailing)

        


        
# blurple = grid_axial(hex_width,hex_size, coordinates=(atlanta_y,atlanta_x))
# blurple.hexes.head()

In [11]:
# blurple.check_randoms(1) 

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.13s/it]


In [19]:
my_hex = blurple.polygons[0]
# for i in range(7651):
#     blurple.polygons.append(my_hex)
blurple.polygons = blurple.polygons[:7651]

len(blurple.polygons),blurple.hexes.shape

(7651, (7651, 4))

In [60]:
blurple.hexes.columns

Index(['hexagon', 'y', 'x', 'transit_score', 'bike_score', 'carshare_score',
       'mass_transit_score', 'ride_hail_score'],
      dtype='object')

In [56]:
blurple.hexes = blurple.hexes[['hexagon','y','x','transit_score']]
blurple.poly_wag()

In [61]:
blurple.hexes[['y', 'x', 'transit_score', 'bike_score', 'carshare_score',
       'mass_transit_score', 'ride_hail_score']].head()

,y,x,transit_score,bike_score,carshare_score,mass_transit_score,ride_hail_score
0,"[33.749, 33.75073205080757, 33.75073205080757,...","[-84.38600000000001, -84.387, -84.389000000000...",90.0,0.11,[0.0],[0.73],[0.16]
1,"[33.747267949192434, 33.749, 33.749, 33.747267...","[-84.38300000000001, -84.384, -84.386000000000...",90.0,0.11,[0.0],[0.73],[0.16]
2,"[33.747267949192434, 33.749, 33.749, 33.747267...","[-84.38900000000001, -84.39, -84.3920000000000...",90.0,0.11,[0.0],[0.73],[0.16]
3,"[33.75246410161514, 33.75419615242271, 33.7541...","[-84.38600000000001, -84.387, -84.389000000000...",90.0,0.11,[0.0],[0.73],[0.16]
4,"[33.75073205080757, 33.75246410161514, 33.7524...","[-84.38900000000001, -84.39, -84.3920000000000...",90.0,0.11,[0.0],[0.73],[0.16]


In [701]:
def hex_round(x,y):
    z = -x -y
    x_d, x = modf(x)
    y_d, y = modf(y)
    z_d, z = modf(z)
    
    if x_d>y_d and x_d>z_d:
        x = -y-z
    elif y_d>z_d:
        y = -x-z
    else:
        z=-x-y
        
    return (int(x),int(y),int(z))

In [702]:
ssq = sqrt(3)/3

def point_loc_to_hex_loc(x_h, y_h, size):
    x = (x_h * 2/3)/ size
    y = ((-x_h/3) + (ssq * y_h)) / size
    return hex_round(x,y)


In [703]:
def hex_center(x_row, y_row, size):
    x = size * 3/2 * x_row
    y = size * (3**0.5) * (y_row + x_row/2)
    return x, y

In [704]:

hex_assign = lambda p: point_loc_to_hex_loc(p.x-washington_x, p.y-washington_y, hex_size)


In [ ]:
df['hexagon'] = df['geometry'].apply(hex_assign)


In [ ]:
df